# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234650924991                   -0.50    7.0         
  2   -7.250176193036       -1.81       -1.40    1.0   7.93ms
  3   -7.251081084543       -3.04       -2.07    1.0   8.09ms
  4   -7.251081084162   +   -9.42       -1.95    2.0   9.88ms
  5   -7.251334602204       -3.60       -2.69    1.0   8.13ms
  6   -7.251338332957       -5.43       -3.22    1.0   8.18ms
  7   -7.251338656465       -6.49       -3.43    2.0   9.72ms
  8   -7.251338789302       -6.88       -3.99    1.0   8.14ms
  9   -7.251338797135       -8.11       -4.56    1.0   8.22ms
 10   -7.251338798517       -8.86       -4.78    3.0   11.6ms
 11   -7.251338798671       -9.81       -5.28    1.0   8.47ms
 12   -7.251338798682      -10.97       -5.29    2.0   10.2ms
 13   -7.251338798701      -10.73       -5.82    1.0   8.54ms
 14   -7.251338798703      -11.60       -5.94    3.0   11.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07375499677555152
[ Info: Arnoldi iteration step 2: normres = 0.549908500030201
[ Info: Arnoldi iteration step 3: normres = 0.846247482904393
[ Info: Arnoldi iteration step 4: normres = 0.23257508343177521
[ Info: Arnoldi iteration step 5: normres = 0.5011997917170732
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.46e-02, 5.23e-02, 4.15e-01, 2.74e-01, 1.63e-02)
[ Info: Arnoldi iteration step 6: normres = 0.29962022542407957
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.96e-03, 6.45e-02, 2.58e-01, 1.02e-01, 9.12e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09783978978077605
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.90e-04, 7.62e-03, 2.29e-02, 6.08e-02, 6.07e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11296665251765914
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.44e-0